### Seção 1 – Importações e Configuração Inicial

In [1]:
# Importações

In [2]:
import os
os.chdir('..')


### Seção 2 – Função para detectar encoding

In [3]:

def detectar_encoding(caminho_arquivo, n_bytes=10000):
    with open(caminho_arquivo, 'rb') as f:
        resultado = chardet.detect(f.read(n_bytes))
    return resultado['encoding']


### Seção 3 – Definir caminhos dos arquivos e detectar encoding

In [6]:
arquivos = {
    'ama': 'D:\PÓS MDE\Seminario_VD\data\\ama_escola.csv',
    'escolas': 'data/escolas_es.csv',
    'idebes': 'data/idebes_por_escolas.csv',
    'olimpiada': 'data/olimpiada_escolas_com_inep.csv',
    'municipios_regionais': 'data/municipios_por_regional.csv'
}

encodings = {}
for nome, caminho in arquivos.items():
    encodings[nome] = detectar_encoding(caminho)

print(encodings)


{'ama': 'UTF-8-SIG', 'escolas': 'UTF-8-SIG', 'idebes': 'ascii', 'olimpiada': 'utf-8', 'municipios_regionais': 'utf-8'}


### Seção 4 – Leitura das bases e renomeação de colunas

In [7]:

# AMA
df_ama = pd.read_csv(arquivos['ama'], encoding=encodings['ama'])
df_ama = df_ama[['CD_ESCOLA', 'NU_ACERTO', 'NU_TOTAL', 'TX_ACERTO']].rename(columns={
    'CD_ESCOLA': 'cod_inep',
    'NU_ACERTO': 'ama_nu_acerto',
    'NU_TOTAL': 'ama_nu_total',
    'TX_ACERTO': 'ama_tx_acerto'
})

# Escolas
df_escolas = pd.read_csv(arquivos['escolas'], encoding=encodings['escolas'])
df_escolas = df_escolas[['Código INEP', 'Escola', 'Município', 'Latitude', 'Longitude']].rename(columns={
    'Código INEP': 'cod_inep',
    'Escola': 'nome_escola',
    'Município': 'municipio',
    'Latitude': 'latitude',
    'Longitude': 'longitude'
})

# IDEBES
df_idebes = pd.read_csv(arquivos['idebes'], encoding=encodings['idebes'], sep=';')
df_idebes = df_idebes[['COD_INEP', 'IDEBES_2024']].rename(columns={
    'COD_INEP': 'cod_inep',
    'IDEBES_2024': 'idebes'
})

# Olimpíada
df_olimpiada = pd.read_csv(arquivos['olimpiada'], encoding=encodings['olimpiada'])
df_olimpiada = df_olimpiada[['cod_inep', 'med_escola']].rename(columns={
    'med_escola': 'olimpiada'
})
df_municipios_regionais = pd.read_csv('data/municipios_por_regional.csv', encoding=encodings['municipios_regionais'])

### Seção 5 – Simulação da base de Iniciação Científica (PICMat)

In [8]:

df_picmat = df_escolas[['cod_inep', 'nome_escola']].copy()
df_picmat['participante'] = 0
df_picmat.loc[df_picmat.sample(frac=0.2, random_state=42).index, 'participante'] = 1


### Seção 6 – Merge progressivo das bases

In [9]:

df_base = df_escolas.copy()

df_base = df_base.merge(df_idebes, on='cod_inep', how='left')
df_base = df_base.merge(df_ama, on='cod_inep', how='left')
df_base = df_base.merge(df_olimpiada, on='cod_inep', how='left')
df_base = df_base.merge(df_picmat[['cod_inep', 'participante']], on='cod_inep', how='left')


In [10]:

# Normalizar o nome dos municípios nas duas bases antes de juntar
df_base['municipio'] = df_base['municipio'].str.upper().str.strip()
df_municipios_regionais['NM_MUN'] = df_municipios_regionais['NM_MUN'].str.upper().str.strip()

# Fazer o merge
df_base = df_base.merge(df_municipios_regionais[['NM_MUN', 'regional']], left_on='municipio', right_on='NM_MUN', how='left')

# Depois pode remover a coluna NM_MUN se quiser
df_base.drop(columns=['NM_MUN'], inplace=True)


### Seção 7 – Limpeza de colunas duplicadas

In [11]:

df_base = df_base.loc[:, ~df_base.columns.duplicated()]


### Seção 8 – Tratamento de valores ausentes

In [12]:

df_base.fillna({
    'idebes': 0,
    'ama_nu_acerto': 0,
    'ama_nu_total': 0,
    'ama_tx_acerto': 0,
    'olimpiada': 0,
    'participante': 0
}, inplace=True)


### Seção 09 – Conferência final


In [13]:

print(df_base.info())
print(df_base.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cod_inep       396 non-null    int64  
 1   nome_escola    396 non-null    object 
 2   municipio      396 non-null    object 
 3   latitude       396 non-null    object 
 4   longitude      396 non-null    object 
 5   idebes         396 non-null    object 
 6   ama_nu_acerto  396 non-null    float64
 7   ama_nu_total   396 non-null    float64
 8   ama_tx_acerto  396 non-null    float64
 9   olimpiada      396 non-null    float64
 10  participante   396 non-null    int64  
 11  regional       396 non-null    object 
dtypes: float64(4), int64(2), object(6)
memory usage: 37.3+ KB
None
   cod_inep                     nome_escola               municipio  \
0  32000057  EEEFM SEBASTIAO COIMBRA ELIZEU      ÁGUA DOCE DO NORTE   
1  32000480          EEEFM OLEGARIO MARTINS      ÁGUA DOCE DO NORTE  

#### Seção 09.1 - Ajuste


In [14]:
df_base['idebes'] = df_base['idebes'].astype(str).str.replace(',', '.').str.strip().astype(float)

### Seção 10 – Exportação da base final

In [16]:
df_base.to_csv('data/dados_escolas.csv', index=False, encoding='utf-8-sig')

b